## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Navahrudak,BY,53.6000,25.8333,40.96,71,31,7.96,scattered clouds
1,1,Busselton,AU,-33.6500,115.3333,53.49,66,0,12.28,clear sky
2,2,Ancud,CL,-41.8697,-73.8203,53.56,92,84,12.33,broken clouds
3,3,Bluff,NZ,-46.6000,168.3333,60.80,81,84,7.49,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,48.06,95,75,5.75,light intensity shower rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Bluff,NZ,-46.6000,168.3333,60.80,81,84,7.49,broken clouds
5,5,Jamestown,US,42.0970,-79.2353,62.53,54,0,5.99,clear sky
7,7,Ponta Do Sol,PT,32.6667,-17.1000,64.00,75,51,4.27,broken clouds
8,8,Laguna,US,38.4210,-121.4238,77.43,40,0,13.80,clear sky
9,9,Ariquemes,BR,-9.9133,-63.0408,76.01,85,39,3.36,scattered clouds
12,12,Navirai,BR,-23.0650,-54.1906,64.36,82,96,3.76,overcast clouds
13,13,Puerto Ayora,EC,-0.7393,-90.3518,75.16,87,100,8.72,overcast clouds
15,15,Port Alfred,ZA,-33.5906,26.8910,71.78,47,1,5.91,clear sky
17,17,Axim,GH,4.8699,-2.2405,80.78,85,43,7.29,scattered clouds
19,19,Touros,BR,-5.1989,-35.4608,77.25,90,100,10.65,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                387
City                   387
Country                379
Lat                    387
Lng                    387
Max Temp               387
Humidity               387
Cloudiness             387
Wind Speed             387
Current Description    387
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
print (preferred_cities_df)

     City_ID          City Country      Lat       Lng  Max Temp  Humidity  \
3          3         Bluff      NZ -46.6000  168.3333     60.80        81   
5          5     Jamestown      US  42.0970  -79.2353     62.53        54   
7          7  Ponta Do Sol      PT  32.6667  -17.1000     64.00        75   
8          8        Laguna      US  38.4210 -121.4238     77.43        40   
9          9     Ariquemes      BR  -9.9133  -63.0408     76.01        85   
..       ...           ...     ...      ...       ...       ...       ...   
674      674        Kalabo      ZM -14.9700   22.6814     66.40        60   
675      675       Leticia      CO  -4.2153  -69.9406     78.85        89   
679      679     Blackfoot      US  43.1905 -112.3450     70.05        43   
680      680          Palu      ID  -0.8917  119.8707     78.69        91   
682      682        Bengbu      CN  32.9408  117.3608     66.94        67   

     Cloudiness  Wind Speed Current Description  
3            84        7.

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bluff,NZ,60.80,broken clouds,-46.6000,168.3333,
5,Jamestown,US,62.53,clear sky,42.0970,-79.2353,
7,Ponta Do Sol,PT,64.00,broken clouds,32.6667,-17.1000,
8,Laguna,US,77.43,clear sky,38.4210,-121.4238,
9,Ariquemes,BR,76.01,scattered clouds,-9.9133,-63.0408,
12,Navirai,BR,64.36,overcast clouds,-23.0650,-54.1906,
13,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
15,Port Alfred,ZA,71.78,clear sky,-33.5906,26.8910,
17,Axim,GH,80.78,scattered clouds,4.8699,-2.2405,
19,Touros,BR,77.25,overcast clouds,-5.1989,-35.4608,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

hotel_names = []

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    print(row.City)
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
 
    
    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
#     reponse = requests.get(base_url, params=params).json()

    
    #6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        
hotel_df["Hotel Name"] = hotel_names
hotel_df.head(10)
        

Bluff
Jamestown
Ponta Do Sol
Laguna
Ariquemes
Navirai
Puerto Ayora
Port Alfred
Axim
Touros
Ribeira Grande
Bartlesville
Bengkulu
Rikitea
Saint-Philippe
Zhenjiang
Kapaa
Riyadh
Am Timan
Nieuw Amsterdam
Nelson Bay
Mataura
Chitral
Coronel Fabriciano
Santa Cruz Cabralia
Hilo
Salalah
Albany
Hotel not found... skipping.
Kahului
Pathein
Avera
Hotel not found... skipping.
Birin
Port Elizabeth
Casper
Butaritari
Motril
Acurenam
Hotel not found... skipping.
Mar Del Plata
Acarau
Kesinga
Vaini
Monrovia
Sao Filipe
Dimbokro
Dourados
Mahebourg
Faanui
Lagoa
Dekar
Hotel not found... skipping.
Bathsheba
Bethel
Paraiba Do Sul
Vilhena
Atuona
Port Macquarie
Mayumba
Marakkanam
Pouebo
Duluth
Katobu
Birao
Hotel not found... skipping.
Kavieng
Coruripe
Puerto Guzman
Ouesso
Banda Aceh
Weiser
Itarema
Capitan Bado
Hithadhoo
Viedma
East London
Pokhara
Black Forest
Carnarvon
Macau
Texarkana
Tura
Balabac
Mackay
Pangkalanbuun
Buala
Faya
Victoria
Panama City
Shahrud
Fare
Salinas
Tambura
Dallas
Guerrero Negro
Amahai
Hamilt

ValueError: Length of values (0) does not match length of index (379)

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Bluff,NZ,60.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
5,Jamestown,US,62.53,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Ponta Do Sol,PT,64.00,broken clouds,32.6667,-17.1000,Hotel do Campo
8,Laguna,US,77.43,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
9,Ariquemes,BR,76.01,scattered clouds,-9.9133,-63.0408,Hotel Sol Nascente - Ariquemes - RO
...,...,...,...,...,...,...,...
674,Kalabo,ZM,66.40,scattered clouds,-14.9700,22.6814,Sikakubete Guest House Kalabo Town Council
675,Leticia,CO,78.85,scattered clouds,-4.2153,-69.9406,Hotel Amazon Bed and Breakfast / Hotel Amazon B&B
679,Blackfoot,US,70.05,overcast clouds,43.1905,-112.3450,Super 8 by Wyndham Blackfoot/Idaho Falls
680,Palu,ID,78.69,overcast clouds,-0.8917,119.8707,Hotel Santika Palu


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
preferred_cities_df.to_csv(output_data_file, index_label="City_ID")
hotel_df.to_csv(output_data_file, index_label="Hotels2")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}


popups = []

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
popups.append(info_box_template)

fig = gmaps.figure(layout=figure_layout, map_type='HYBRID')


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = preferred_cities_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
current_description = ["Current Weather"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))